# Câu hỏi
Liệu ta có thể dự đoán được 'Score' của một bộ anime hay không nếu dựa vào các thông tin nội tại của bộ anime đó? 

Ta sẽ chỉ dựa vào các đặc trưng: <span style="color: yellow;">'Genres', 'Synopsis', 'Type', 'Episodes', 'Producers', 'Studios', 'Source', 'Time per ep (Min)', 'Released date', 'Completed date', 'Rating'.</span>

Ta sẽ không dùng các đặc trưng: <span style="color: red;">'Rank', 'Popularity', 'Favorites', 'Scored By', 'Members'</span>. Đặc trưng <span style="color: red;">'Rating'</span> bản chất được suy ra từ <span style="color: yellow;">'Score'</span> nên ta sẽ bỏ. Các đặc trưng còn lại bỏ do chúng có mối quan hệ mạnh với đặc trưng <span style="color: yellow;">'Score'</span>, chúng phản ảnh sự đón nhận của khán giả đối với bộ phim. Ở đây ta muốn tìm hiểu mối quan hệ giữa các đặc trưng nội tại như Thể loại, Số tập, Nhà sản xuất,... đối với Điểm số của bộ phim, nếu thêm các đặc trưng kia vào thì sẽ mất đi ý nghĩa.

Đồng thời ta cũng sẽ bỏ đặc trưng <span style="color: red;">'Name'</span>, vì tên phim khá ngẫu nhiên và thường không có tác dụng dự đoán điều gì.

---

# 1. Đọc dữ liệu

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re

import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\trinh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\trinh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Các PATH dùng trong bài này

In [23]:
ROOT = os.path.dirname(os.getcwd())
DATASET_FOLDER = os.path.join(ROOT, 'Data Preprocessing')
ANIME_CLEAN_DATASET = os.path.join(DATASET_FOLDER, 'anime-data-preprocessing.csv')

Đọc dữ liệu vào dataframe

In [24]:
df = pd.read_csv(ANIME_CLEAN_DATASET)
print(df.info())
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13834 entries, 0 to 13833
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               13834 non-null  object 
 1   Score              13834 non-null  float64
 2   Genres             13834 non-null  object 
 3   Synopsis           13834 non-null  object 
 4   Type               13834 non-null  object 
 5   Episodes           13834 non-null  float64
 6   Producers          13834 non-null  object 
 7   Studios            13834 non-null  object 
 8   Source             13834 non-null  object 
 9   Time per ep (Min)  13834 non-null  float64
 10  Rating             13834 non-null  object 
 11  Rank               13834 non-null  int64  
 12  Popularity         13834 non-null  int64  
 13  Favorites          13834 non-null  int64  
 14  Scored By          13834 non-null  int64  
 15  Members            13834 non-null  int64  
 16  Released date      138

,Name,Score,Genres,Synopsis,Type,Episodes,Producers,Studios,Source,Time per ep (Min),Rating,Rank,Popularity,Favorites,Scored By,Members,Released date,Completed date
0,Fullmetal Alchemist: Brotherhood,9.10,"Action, Adventure, Drama, Fantasy",After a horrific alchemy experiment goes wrong...,TV,64.0,"Aniplex, Square Enix, Mainichi Broadcasting Sy...",Bones,Manga,24.00,R - 17+ (violence & profanity),1,3,217606,2020030,3176556,2009-04-05,2010-07-04
1,Steins;Gate,9.07,"Drama, Sci-Fi, Suspense",Eccentric scientist Rintarou Okabe has a never...,TV,24.0,"Frontier Works, Media Factory, Kadokawa Shoten...",White Fox,Visual novel,24.00,PG-13 - Teens 13 or older,2,13,182964,1336233,2440369,2011-04-06,2011-09-14
2,Bleach: Sennen Kessen-hen,9.07,"Action, Adventure, Fantasy",Substitute Soul Reaper Ichigo Kurosaki spends ...,TV,13.0,"TV Tokyo, Aniplex, Dentsu, Shueisha",Pierrot,Manga,24.00,R - 17+ (violence & profanity),3,464,17999,213872,445198,2022-10-11,2022-12-27
3,Gintama°,9.06,"Action, Comedy, Sci-Fi","Gintoki, Shinpachi, and Kagura return as the f...",TV,51.0,"TV Tokyo, Aniplex, Dentsu",Bandai Namco Pictures,Manga,24.00,PG-13 - Teens 13 or older,4,331,15947,237957,595767,2015-04-08,2016-03-30
4,Shingeki no Kyojin Season 3 Part 2,9.05,"Action, Drama",Seeking to restore humanity's diminishing hope...,TV,10.0,"Production I.G, Dentsu, Mainichi Broadcasting ...",Wit Studio,Manga,23.00,R - 17+ (violence & profanity),5,24,55245,1471825,2104016,2019-04-29,2019-07-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13829,Kokuhaku,2.30,Horror,"Beside a certain building, a girl appears to m...",ONA,1.0,UNKNOWN,UNKNOWN,Original,0.67,R - 17+ (violence & profanity),13830,7105,17,4904,6697,2015-08-27,2015-08-27
13830,Hametsu no Mars,2.22,"Horror, Sci-Fi",Several months after a probe returning from Ma...,OVA,1.0,"Idea Factory, King Records, Design Factory",WAO World,Visual novel,19.00,R - 17+ (violence & profanity),13831,2512,295,47630,65622,2005-07-06,2005-07-06
13831,Tsui no Sora,2.22,Hentai,"After the sudden death of a student, mysteriou...",OVA,1.0,Obtain Future,UNKNOWN,Visual novel,23.00,Rx - Hentai,13832,7563,24,3436,5713,2002-08-10,2002-08-10
13832,Utsu Musume Sayuri,1.98,"Avant Garde, Comedy",Sayuri is a curious creature who lives on her ...,OVA,1.0,UNKNOWN,UNKNOWN,Original,3.00,R+ - Mild Nudity,13833,4492,50,15873,20789,2003-01-01,2003-01-01


Ta sẽ bỏ đi các đặc trưng <span style="color: red;">'Rank', 'Popularity', 'Favorites', 'Scored By', 'Members'</span>. 

In [25]:
columns_to_be_dropped = ['Name', 'Rank', 'Popularity', 'Favorites', 'Scored By', 'Members']
df.drop(columns=columns_to_be_dropped, inplace=True)
df.head(3)

,Score,Genres,Synopsis,Type,Episodes,Producers,Studios,Source,Time per ep (Min),Rating,Released date,Completed date
0,9.10,"Action, Adventure, Drama, Fantasy",After a horrific alchemy experiment goes wrong...,TV,64.0,"Aniplex, Square Enix, Mainichi Broadcasting Sy...",Bones,Manga,24.0,R - 17+ (violence & profanity),2009-04-05,2010-07-04
1,9.07,"Drama, Sci-Fi, Suspense",Eccentric scientist Rintarou Okabe has a never...,TV,24.0,"Frontier Works, Media Factory, Kadokawa Shoten...",White Fox,Visual novel,24.0,PG-13 - Teens 13 or older,2011-04-06,2011-09-14
2,9.07,"Action, Adventure, Fantasy",Substitute Soul Reaper Ichigo Kurosaki spends ...,TV,13.0,"TV Tokyo, Aniplex, Dentsu, Shueisha",Pierrot,Manga,24.0,R - 17+ (violence & profanity),2022-10-11,2022-12-27


Hai đặc trưng Released date và Completed date vẫn chưa được chuyển về dạng datetime nên ta sẽ chuyển ở đây luôn, các bước xử lý phức tạp hơn ta làm trong phần sau.

In [26]:
df['Released date'] = pd.to_datetime(df['Released date'], errors='coerce')
df['Completed date'] = pd.to_datetime(df['Completed date'], errors='coerce')

---

# 2. Tiền xử lý dữ liệu 

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13834 entries, 0 to 13833
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Score              13834 non-null  float64       
 1   Genres             13834 non-null  object        
 2   Synopsis           13834 non-null  object        
 3   Type               13834 non-null  object        
 4   Episodes           13834 non-null  float64       
 5   Producers          13834 non-null  object        
 6   Studios            13834 non-null  object        
 7   Source             13834 non-null  object        
 8   Time per ep (Min)  13834 non-null  float64       
 9   Rating             13834 non-null  object        
 10  Released date      13834 non-null  datetime64[ns]
 11  Completed date     13727 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(3), object(7)
memory usage: 1.3+ MB


Hiện tại dữ liệu của chúng ta gồm 11 feature (không tính 'Score').

- Các feature dạng object gồm: <span style="color: yellow;">'Genres', 'Synopsis', 'Type', 'Producers', 'Studios', 'Source', 'Rating'</span>.

- Các feature dạng số gồm: <span style="color: yellow;">'Episodes', 'Time per ep (Min)'</span>.

- Các feature dạng datetime gồm: <span style="color: yellow;">'Released date', 'Completed date'</span>.

Ta cần để ý các vấn đề quan trọng sau:
- Ba feature <span style="color: yellow;">'Genres'</span>, <span style="color: yellow;">'Studios'</span> và <span style="color: yellow;"> 'Producers'</span> có dữ liệu kiểu: 'Action, Drama, Fantasy' hay 'TV Tokyo, Aniplex', tức là categorical đa nhãn. 
- Ba feature <span style="color: yellow;">'Type'</span>, <span style="color: yellow;">'Source'</span> và <span style="color: yellow;">'Rating'</span> là kiểu categorical đơn nhãn.
- Feature <span style="color: yellow;">'Synopsis'</span> là một đoạn text (sẽ xử lý kiểu bag of words/tfidf).
- Feature <span style="color: yellow;">'Completed date'</span> có lẫn một số giá trị NaT, tức bộ anime vẫn đang chiếu.
- Ba feature object <span style="color: yellow;">'Producers'</span>, <span style="color: yellow;">'Studios'</span> và <span style="color: yellow;">'Source'</span> vẫn còn lẫn một số giá trị 'UNKNOWN'.
- Feature <span style="color: yellow;">'Synopsis'</span> vẫn còn lẫn một số giá trị 'No description available for this anime.'

### 2.1 Xử lý các feature còn lẫn 'UNKNOWN'

In [28]:
# Kiểm tra số lượng giá trị 'UNKNOWN' có trong một cột
def count_unknown_values(column):
    return column.str.upper().str.contains('UNKNOWN').sum()

# Lưu ý ở đây ta biết rõ chỉ còn 2 feature 'Producers', 'Studios', 'Source' là có giá trị 'UNKNOWN'
unknown_count = df[['Producers', 'Studios', 'Source']].apply(count_unknown_values)
unknown_count

Producers    5018
Studios      2345
Source       1508
dtype: int64

- Do feature <span style="color: yellow;"> 'Producers'</span> có quá nhiều giá trị thiếu ('UNKNOWN'), ta xem xét bỏ luôn feature đó ra khỏi dataset.

- Với hai feature còn lại ta sẽ drop các sample chứa giá trị thiếu

In [29]:
# Drop the 'Producers' column
df.drop(columns=['Producers'], inplace=True)

# Drop các sample có giá trị 'UNKNOWN' trong 'Studios' và 'Source'
mask = df['Studios'].str.upper().str.contains('UNKNOWN') | df['Source'].str.upper().str.contains('UNKNOWN')
df = df[~mask]
df = df.reset_index(drop=True)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10562 entries, 0 to 10561
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Score              10562 non-null  float64       
 1   Genres             10562 non-null  object        
 2   Synopsis           10562 non-null  object        
 3   Type               10562 non-null  object        
 4   Episodes           10562 non-null  float64       
 5   Studios            10562 non-null  object        
 6   Source             10562 non-null  object        
 7   Time per ep (Min)  10562 non-null  float64       
 8   Rating             10562 non-null  object        
 9   Released date      10562 non-null  datetime64[ns]
 10  Completed date     10468 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(3), object(6)
memory usage: 907.8+ KB


### 2.1 Xử lý các feature categorical đơn nhãn

Đối với 2 đặc trưng là <span style="color: yellow;">'Type'</span> và <span style="color: yellow;">'Source'</span>, do chúng không có <span style="color: pink;">Tính thứ tự</span> nên ta dùng <span style="color: orange;">One-hot encoding</span> (Ở đây dùng sẵn hàm <span style="color: orange;">Dummy encoding</span> của pandas cho tiện)

In [31]:
type_dummies = df['Type'].str.get_dummies()
source_dummies = df['Source'].str.get_dummies()

# Thêm dữ liệu mới tạo vào df cũ
df = pd.concat([df, type_dummies, source_dummies], axis=1)

# Drop các cột 'Type', 'Source' sau khi đã tạo dummies
df.drop(columns=['Type', 'Source'], inplace=True)

Đối với đặc trưng <span style="color: yellow;">'Rating'</span>, do chúng có <span style="color: pink;">Tính thứ tự</span> nên ta sẽ dùng <span style="color: orange;">Label encoding</span>

In [32]:
# Tạo encoder
encoder = LabelEncoder()
df['Rating'] = encoder.fit_transform(df['Rating'])

Check lại

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10562 entries, 0 to 10561
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Score              10562 non-null  float64       
 1   Genres             10562 non-null  object        
 2   Synopsis           10562 non-null  object        
 3   Episodes           10562 non-null  float64       
 4   Studios            10562 non-null  object        
 5   Time per ep (Min)  10562 non-null  float64       
 6   Rating             10562 non-null  int32         
 7   Released date      10562 non-null  datetime64[ns]
 8   Completed date     10468 non-null  datetime64[ns]
 9   Movie              10562 non-null  int64         
 10  Music              10562 non-null  int64         
 11  ONA                10562 non-null  int64         
 12  OVA                10562 non-null  int64         
 13  Special            10562 non-null  int64         
 14  TV    

### 2.2 Xử lý các feature categorical đa nhãn

Ta chuyển dữ liệu thành một vector chứa các giá trị nhị phân

Ví dụ nếu bài toán có tổng cộng 3 class: Dog, Cat, Bird. Một dữ liệu có giá trị 'Dog, Cat' sẽ được biểu diễn bằng vector [1, 1, 0]

In [34]:
# Trước tiên ta cần tách các giá trị string thành một list các class
df['Genres'] = df['Genres'].apply(lambda x: [x.strip() for x in x.split(',')])
df['Studios'] = df['Studios'].apply(lambda x: [x.strip() for x in x.split(',')])

In [35]:
# Sử dụng mulitlabel binarizer của sklearn
mlbinarizer = MultiLabelBinarizer()


# Feature Studios
# fit_transform cột Studios thành một ma trận nhị phân
altered_Studios_matrix = mlbinarizer.fit_transform(df['Studios'])
# Chuyển ma trận đó thành DataFrame
altered_Studios_df = pd.DataFrame(altered_Studios_matrix, columns=mlbinarizer.classes_)
# Gắn dataframe mới vào dataframe cũ
df = pd.concat([df, altered_Studios_df], axis=1)


mlbinarizer = MultiLabelBinarizer()
# Feature Genres
# fit_transform cột Genres thành một ma trận nhị phân
altered_Genres_matrix = mlbinarizer.fit_transform(df['Genres'])
# Chuyển ma trận đó thành DataFrame
altered_Genres_df = pd.DataFrame(altered_Genres_matrix, columns=mlbinarizer.classes_)
# Gắn dataframe mới vào dataframe cũ
df = pd.concat([df, altered_Genres_df], axis=1)

In [36]:
# Xóa các cột 'Genres', 'Studios' 
df.drop(columns=['Genres', 'Studios'], inplace=True)

Check lại

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10562 entries, 0 to 10561
Columns: 900 entries, Score to Suspense
dtypes: datetime64[ns](2), float64(3), int32(872), int64(22), object(1)
memory usage: 37.4+ MB


### 2.3 Xử lý các feature datetime

Để cho đơn giản, đối với hai đặc trưng <span style="color: yellow;">'Released date'</span> và <span style="color: yellow;">'Completed date'</span>, ta sẽ chỉ giữ lại phần <span style="color: yellow;">năm</span>. 

Với 'Completed date' nếu gặp dữ liệu dạng NaT (bộ phim này vẫn đang chiếu), ta sẽ gán năm là 2023, tức năm mà bộ dataset này được tạo ra.

In [38]:
df['Released date'] = df['Released date'].dt.year

df['Completed date'] = df['Completed date'].dt.year
df['Completed date'] = df['Completed date'].fillna(2023)

# Chuyển về kiểu int cho đồng bộ
df['Released date'] = df['Released date'].astype(int)
df['Completed date'] = df['Completed date'].astype(int)

### 2.4 Xử lý feature dạng text

Chỉ có duy nhất một đặc trưng là <span style="color: yellow;">'Synopsis'</span>.

Ta sẽ thực hiện <span style="color: pink;">tiền xử lý text</span> sau đó <span style="color: pink;">chuyển chúng thành vector</span>

In [39]:
# def preprocessing_text(text):
#     # Chuyển về dạng chữ thường
#     text = text.lower()

#     # Bỏ dấu câu
#     translator = str.maketrans('', '', string.punctuation)
#     text = text.translate(translator)

#     # Tokenize tạm thời để xử lý 
#     tokens = nltk.word_tokenize(text)

#     # Stemming
#     stemmer = nltk.stem.PorterStemmer() 
#     tokens = [stemmer.stem(token) for token in tokens]

#     # Xóa stop words
#     stopwords = nltk.corpus.stopwords.words('english')
#     tokens = [token for token in tokens if token not in stopwords]

#     return ' '.join(tokens).strip()

# # Apply hàm preprocessing_text cho cột 'Synopsis'
# df['Synopsis'] = df['Synopsis'].apply(preprocessing_text)

In [40]:
# # Tạo tfidf vectorizer
# vectorizer = TfidfVectorizer()

# tfidf_data = vectorizer.fit_transform(df['Synopsis'])
# tfidf_dataframe = pd.DataFrame(tfidf_data.toarray(), columns=vectorizer.get_feature_names_out())

# # Gắn dataframe mới vào dataframe cũ
# df = pd.concat([df, tfidf_dataframe], axis=1)

# # Bỏ cột 'Synopsis' sau khi đã xử lý
# df.drop(columns=['Synopsis'], inplace=True)

---

# 3. Huấn luyện mô hình

Chia X, y. Chia train-test

In [41]:
X = df.drop(columns=['Score', 'Synopsis'])
y = df['Score']

In [42]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=22
)

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [43]:
# Dự đoán trên tập test
y_pred = rf_model.predict(X_test)

# Tính các chỉ số hồi quy
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R² Score: {r2}")


Mean Absolute Error (MAE): 0.4516120621108661
Mean Squared Error (MSE): 0.35435901686207005
R² Score: 0.489072305780872
